### notebook for learning event handler system

In [1]:
from mesmerize_core import *
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from fastplotlib import GridPlot, Image, Plot, Line, Heatmap
from scipy.spatial import distance
from ipywidgets.widgets import IntSlider, VBox
import pygfx

In [2]:
class Contour_Selection():
    def __init__(
        self,
        gp: GridPlot,
        coms,
    ):
        self.gp = gp
        self.heatmap = self.gp.subplots[0, 1].scene.children[0]
        self.image = None
        self._contour_index = None 
        
        for child in self.gp.subplots[0, 0].scene.children:
            if isinstance(child, pygfx.Image):
                self.image = child
                break;
        if self.image == None:
            raise ValueError("No image found!")
        self.coms = np.array(coms)
    
        self.image.add_event_handler(self.event_handler, "click")
    
    # first need to add event handler for when contour is clicked on
    # should also trigger highlighting in heatmap
    def event_handler(self, event):
        if self._contour_index is not None:
            self.remove_highlight()
            self.add_highlight(event)
        else:
            self.add_highlight(event)
    
    def add_highlight(self, event):
        click_location = np.array(event.pick_info["index"])
        self._contour_index = np.linalg.norm((self.coms - click_location), axis=1).argsort()[0] + 1
        line = self.gp.subplots[0, 0].scene.children[self._contour_index]
        line.geometry.colors.data[:] = np.array([1.0, 1.0, 1.0, 1.0]) 
        line.geometry.colors.update_range()
        #self.heatmap.add_highlight(self._contour_index)
    
    def remove_highlight(self):
        # change color of highlighted index back to normal
        line = self.gp.subplots[0, 0].scene.children[self._contour_index]
        line.geometry.colors.data[:] = np.array([1., 0., 0., 0.7]) 
        line.geometry.colors.update_range()
        # for h in self.heatmap._highlights:
        #     self.heatmap.remove_highlight(h)

In [3]:
set_parent_raw_data_path("/home/kushal/caiman_data/")

batch_path = "/home/clewis7/caiman_data/cnmf_practice/batch.pickle"

movie_path = "/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif"

In [4]:
df = load_batch(batch_path)

In [5]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': 1ed8feb3-9fc8-4a78-8f...,None,1ed8feb3-9fc8-4a78-8f6d-164620822016
1,cnmf,my_movie,1ed8feb3-9fc8-4a78-8f6d-164620822016/1ed8feb3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 5f4e3e27-ac1f-4ede-90...,None,5f4e3e27-ac1f-4ede-903b-be43bd81fddc


In [6]:
gp = GridPlot(shape=(1,2))

contours, coms = df.iloc[-1].cnmf.get_contours()
movie = df.iloc[-1].cnmf.get_input_memmap()
temporal = df.iloc[-1].cnmf.get_temporal()

contour_graphic =  Image(movie[0].T, cmap="gnuplot2")
heatmap = Heatmap(data=temporal[:,0:1000], cmap="jet")

slider = IntSlider(value=0, min=0, max=movie.shape[0] - 1, step=1)

gp.subplots[0,0].add_graphic(contour_graphic)
gp.subplots[0,1].add_graphic(heatmap)

for coor in contours:
    # line data has to be 3D
    zs = np.ones(coor.shape[0])  # this will place it above the image graphic
    c3d = [coor[:, 0], coor[:, 1], zs]
    coors_3d = np.dstack(c3d)[0]

    # make all the lines red, [R, G, B, A] array
    colors = np.vstack([[1., 0., 0., 0.7]] * coors_3d.shape[0])
    line_graphic = Line(data=coors_3d, colors=colors, zlevel=1)
    gp.subplots[0, 0].add_graphic(line_graphic)

previous_slider_value = 0
def update_frame(): 
    if slider.value == previous_slider_value:
        return
    contour_graphic.update_data(data=movie[slider.value].T)

gp.add_animations([update_frame])

VBox([gp.show(), slider])

RFBOutputContext()

In [7]:
contour_selection = Contour_Selection(gp=gp, coms=coms)

In [11]:
temporal.shape

(155, 3000)

In [21]:
type(coms[0])

numpy.ndarray

In [22]:
coms[0]

array([7.12861818, 9.84114483])

In [16]:
gp.subplots[0, 1]

next steps:
    clicking on a contour should highlight it and the heatmap row